In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import math
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
ticker = 'GOOG'
period1 = int(time.mktime(datetime.datetime(2000, 1, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2024, 1, 1, 23, 59).timetuple()))
interval = '1wk' #1d, 1m

In [3]:
query = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'

In [4]:
df = pd.read_csv(query)

df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-16,2.490664,2.716817,2.390042,2.697639,2.694573,1.356285e+09
1,2004-08-23,2.758411,2.826406,2.579581,2.643840,2.640835,1.125298e+09
2,2004-08-30,2.622171,2.627402,2.464263,2.490913,2.488082,7.942379e+08
3,2004-09-06,2.515820,2.654052,2.480951,2.623417,2.620435,4.741185e+08
4,2004-09-13,2.655795,2.926282,2.651561,2.926282,2.922956,9.670753e+08


In [5]:
df['HL_percent'] = (df['High'] - df['Close']) / df['Close'] * 100.0
df['Percent_change'] = (df['Close'] - df['Open']) / df['Open'] * 100.0

df = df[['Close', 'HL_percent', 'Percent_change', 'Volume']]

df.head()

,Close,HL_percent,Percent_change,Volume
0,2.697639,0.710918,8.310033,1.356285e+09
1,2.643840,6.905335,-4.153514,1.125298e+09
2,2.490913,5.479477,-5.005699,7.942379e+08
3,2.623417,1.167752,4.276816,4.741185e+08
4,2.926282,0.000000,10.184785,9.670753e+08


In [6]:
forecast_col = 'Close'
df.fillna(-9999, inplace=True)

forecast_out = int(math.ceil(0.01 * len(df)))
print(forecast_out)

df['label'] = df[forecast_col].shift(-forecast_out)

df.dropna(inplace=True)

df.head()

11


,Close,HL_percent,Percent_change,Volume,label
0,2.697639,0.710918,8.310033,1.356285e+09,4.217940
1,2.643840,6.905335,-4.153514,1.125298e+09,4.533009
2,2.490913,5.479477,-5.005699,7.942379e+08,4.219185
3,2.623417,1.167752,4.276816,4.741185e+08,4.468003
4,2.926282,0.000000,10.184785,9.670753e+08,4.493159


In [10]:
X = np.array(df.drop(['label'], 1))
y = np.array(df['label'])

X = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#clf = LinearRegression() # --> -0.101
#clf = LinearRegression(n_jobs=10) # --> 0.028400282642244812
clf = svm.SVR() # --> 0.8273249955157471
#clf = svm.SVR(kernel='poly') # --> 0.7297308393653856
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)

print(accuracy)

0.8387448646893464


C:\Users\prathmesh\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
C:\Users\prathmesh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\preprocessing\_data.py:236: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  "Numerical issues were encountered "
